# Forward Procedure

En este ejemplo, planteamos una forma eficiente de programar el procedimiento de avance a partir de operaciones con arreglos de numpy. En primer lugar, llamamos a la librería de numpy, la cual utilizaremos para crear y operar con los arreglos.

In [1]:
from __future__ import division
import numpy as np

Supóngamos que tenemos un Modelo Oculto de Markov $HMM = (\Sigma,\Delta,A,B,\Pi)$, donde el alfabeto deobservaciones se define por:

$$\Delta = \{D, NN, V\}$$

Y el alfabeto de emisiones está determinado como: 

$$\Sigma = \{la, niña, garza, pasa\}$$

Para determinar estos símbolos, indexaremos los elementos de cada alfabeto a partir de dictionarios (en este caso hace falta únicamente indexar el alfabeto $\Sigma$):

In [2]:
Sidx = {'la':0,'nina':1, 'garza':2, 'pasa':3}

Las probabilidades se pueden determinar de la siguiente forma:

In [3]:
Pi = np.array([4/6, 1/6, 1/6])
A = np.array([[1/6,1/6,1/5],[4/6,1/6,1/5],[1/6,3/6,1/5]])
B = np.array([[4/7,1/7,1/6],[1/7,2/7,1/6],[1/7,2/7,1/6],[1/7,2/7,3/6]])

Estas probabilidades, junto con los alfabetos de observaciones y emisiones nos dan el Modelo Oculto de Markov $HMM$. Ahora supongamos que queremos determinar la probabilidad de una cadena de observaciones $d\in\Delta^*$, definida como sigue:

In [4]:
d = 'la nina pasa la garza'
d = d.split()
print d

['la', 'nina', 'pasa', 'la', 'garza']


Utilizaremos el procedimiento de avance para determinar dicha probabilidad. Realizaremos los pasos de inicialización, inducción y terminación:

(1) Inicialización. La inicialización está dada por el almacenamiento de las probabilidades inciiales en la variable de avance:

$$\alpha_i(0) := \pi_i$$

In [5]:
a = Pi

(2) Inducción. Los siguientes pasos consistirán en ir actualizando la variable de avance a partir de los diferentes estados de la cadena:

$$\alpha_i(t+1) =  p(s_t|d_i)\sum_{j=1}^N p(d_i|d_j)\alpha_j(t)$$

En este caso, utilizaremos operaciones entre matrices y vectores, pues es evidente que (tomando los elementos del modelo $A,B,\Pi$):

$$\alpha_i(t+1) = B_{t,\cdot} \otimes (A_{i,\cdot} \alpha_{\cdot}(t))$$

donde $B_{t,\cdot}$ representa el vector renglón que corresponde a la observación en el estado $t$ y  $\alpha_{\cdot}(t)$ es el vector que contiene todas las variables $\alpha_i(t)$. Aquí, $A_{i,\cdot}$ representa el i-ésimo vector renglón de $A$ y $\otimes$ representa el producto de Hadamard. En general, podemos actualizar todo el vector $\alpha(t+1)$ (cuyas entradas son $\alpha_i(t+1)$) tomando la matriz completa $A$. De tal forma que:

$$\alpha(t+1) = B_{t,\cdot} \otimes (A \alpha_{\cdot}(t)) $$

In [6]:
for t in range(len(d)):
    a =  B[ Sidx[d[t]] ] * np.dot(A,a)
    print 'simbolo:', (' ').join(d[:t+1]), '- probabilidad',a.sum(0)

simbolo: la - probabilidad 0.208597883598
simbolo: la nina - probabilidad 0.0395187704712
simbolo: la nina pasa - probabilidad 0.0112203685707
simbolo: la nina pasa la - probabilidad 0.00208955127865
simbolo: la nina pasa la garza - probabilidad 0.000411172135693


(3) Terminación. Para la terminación, tenemos que:

$$p(d) = \sum_{i=1}^N \alpha_i(T)$$

Pero ya que hemos guardado cada variable $\alpha_i$ como la entrada de un vector, basta sumar las entradas de este vector para obtener esta probabilidad:

In [8]:
print 'Probabilidad de la cadena',d, 'es:', a.sum(0)

 Probabilidad de la cadena ['la', 'nina', 'pasa', 'la', 'garza'] es: 0.000411172135693
